In [10]:
import pandas as pd
import pprint

In [1]:
def load_economy():
    fields = ["state", "county", "Natural Amenity", "Median Child Income", "Gini"]
    df_income = pd.read_csv("../data/county_amenity_income.csv").drop(["Unnamed: 0"], axis=1)[fields]
    df_employ = pd.read_csv("../data/2015_labor_force_data.csv").drop_duplicates(
        subset=["County Name", "State"]
    ).rename(columns={"State": "state", "County Name": "county"}).set_index(["state", "county"])
    df_life = pd.read_csv("../data/US_life_expectancy.csv")
    state_map = pd.read_csv("../data/state_map.csv")
    df_life = df_life[["county", "Life Expectancy", "State"]].groupby(["State", "county"]).mean().reset_index()
    df_life["state"] = df_life["State"].str.lower()
    df_life = df_life.merge(
        state_map,
        on="state"
    ).drop_duplicates(
        subset=["county", "state"]
    ).drop(["State", "state"], axis=1).rename(
        columns={"acronym":"state"}
    ).set_index(["state", "county"])
    df_life = df_life.merge(
        df_employ, left_index=True, right_index=True, how="left"
    )[["Labor Force", "Employed", "Unemployed", "Unemployment Rate", "Life Expectancy"]].round(3).reset_index()
    df_life["county"] = df_life["county"].str.upper()
    df_life = df_life.set_index(["state", "county"])
    df_income["county"] = df_income["county"] + " COUNTY"
    df_income = df_income.set_index(["state", "county"])
    df_economy = df_life.merge(df_income, left_index=True, right_index=True).reset_index()
    for column in df_economy.columns:
        if df_economy[column].dtype != object:
            df_economy[column] = df_economy[column].fillna(df_economy[column].mean())
    df_economy["Gini_bin"] = (df_economy["Gini"] // 0.1 * 0.1).round(1)
    df_economy["Employ_bin"] = df_economy["Unemployment Rate"] // 5 * 5 
    df_economy["Avg Natural Amenity"] = df_economy.set_index(
        ["Gini_bin", "Employ_bin"]
    ).merge(df_economy.groupby(
        ["Gini_bin", "Employ_bin"]
    )[["Natural Amenity"]].mean().rename(
        columns={"Natural Amenity":"Avg Natural Amenity"}), 
        left_index=True, 
        right_index=True
    ).reset_index()["Avg Natural Amenity"]
    return df_economy

def load_bin(df_economy):
    df_bin = pd.DataFrame(df_economy.groupby(
        ["Gini_bin", "Employ_bin"]
    )[["Natural Amenity"]].mean().rename(
        columns={"Natural Amenity":"Avg Natural Amenity"}
    ))
    df_bin["Count"] = pd.DataFrame(df_economy.groupby(
        ["Gini_bin", "Employ_bin"]
    )[["Natural Amenity"]].count().rename(
        columns={"Natural Amenity":"Count"}
    ))
    return df_bin.reset_index()

def load_amenity():
    df_amenity = pd.read_csv("../data/amenity.csv")
    fields = ["Standardized Jan Temperature", "Standardized Jan Sunlight", "Standardized July Temperature", "Standardized July Humidity", "Standardized Topography", "Standardized Water Area", "Natural Amenity"]
    us_rank = df_amenity[fields].rank()
    state_rank = df_amenity.groupby(["state"]).rank()[fields]
    for field in fields:
        df_amenity["{}_state_rank".format(field)] = state_rank[field].max() - state_rank[field] + 1
        df_amenity["{}_state_rank_filter".format(field)] = state_rank[field].mean() - abs(state_rank[field] - state_rank[field].mean())
        df_amenity["{}_us_rank".format(field)] = us_rank[field].max() - us_rank[field] + 1
        df_amenity["{}_us_rank_filter".format(field)] = us_rank[field].mean() - abs(us_rank[field] - us_rank[field].mean())
    return df_amenity


In [95]:
df_amenity = pd.read_csv("../data/df_amenity.csv").drop(["Unnamed: 0"], axis=1)
df_amenity["FIPS"] = df_amenity["id"].astype(int)
df_amenity.drop(["id"], axis=1).to_csv("../data/df_amenity.csv")

In [97]:
df_bin = pd.read_csv("../data/df_bin.csv")
df_bin

,Gini_bin,Employ_bin,Avg Natural Amenity,Count
0,0.1,0.0,0.720000,2
1,0.1,5.0,5.180000,1
2,0.2,0.0,-1.409826,287
3,0.2,5.0,-1.057805,123
4,0.2,10.0,0.196667,3
5,0.3,0.0,-0.471671,706
6,0.3,5.0,0.251351,807
7,0.3,10.0,0.891364,22
8,0.3,15.0,-0.100000,4
9,0.4,0.0,0.681910,199


In [98]:
pd.read_csv("../data/df_economy.csv")

,Unnamed: 0,State,County,Labor Force,Employed,Unemployed,Unemployment Rate,Life Expectancy,Natural Amenity,Median Child Income,Gini,Gini_bin,Employ_bin,Avg Natural Amenity
0,0,AL,AUTAUGA COUNTY,"25,541","24,206","1,335",5.2,75.320,0.78,31900.0,0.381,0.3,5.0,0.720000
1,1,AL,BALDWIN COUNTY,"87,493","82,619","4,874",5.6,77.483,1.82,30200.0,0.491,0.4,5.0,0.720000
2,2,AL,BARBOUR COUNTY,"8,590","7,823",767,8.9,73.971,0.19,22400.0,0.492,0.4,5.0,5.180000
3,3,AL,BIBB COUNTY,"8,552","7,983",569,6.7,73.550,-0.15,26800.0,0.418,0.4,5.0,-1.409826
4,4,AL,BLOUNT COUNTY,"24,416","23,089","1,327",5.4,76.078,0.23,33200.0,0.334,0.3,5.0,-1.409826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2905,2905,WY,SWEETWATER COUNTY,"22,556","21,509","1,047",4.6,79.178,2.63,52800.0,0.257,0.2,0.0,2.716667
2906,2906,WY,TETON COUNTY,"14,574","14,016",558,3.8,81.950,5.39,44100.0,0.614,0.6,0.0,2.716667
2907,2907,WY,UINTA COUNTY,"9,593","9,120",473,4.9,77.700,3.32,55000.0,0.228,0.2,0.0,-0.240000
2908,2908,WY,WASHAKIE COUNTY,"4,233","4,058",175,4.1,80.600,1.10,45800.0,0.379,0.3,0.0,-0.310000


In [106]:
df_amenity = pd.read_csv("../data/df_amenity.csv").drop(["Unnamed: 0"], axis=1)
fields = ["Standardized Jan Temperature", "Standardized Jan Sunlight", "Standardized July Temperature", "Standardized July Humidity", "Standardized Topography", "Standardized Water Area", "Natural Amenity"]
us_rank = df_amenity[fields].rank()
state_rank = df_amenity.groupby(["State"]).rank()[fields]
for field in fields:
    df_amenity["{}_state_rank".format(field)] = state_rank[field].max() - state_rank[field] + 1
    df_amenity["{}_state_rank_filter".format(field)] = state_rank[field].mean() - abs(state_rank[field] - state_rank[field].mean())
    df_amenity["{}_us_rank".format(field)] = us_rank[field].max() - us_rank[field] + 1
    df_amenity["{}_us_rank_filter".format(field)] = us_rank[field].mean() - abs(us_rank[field] - us_rank[field].mean())
df_amenity.to_csv("../data/df_amenity.csv")

In [118]:
df_amenity["Natural Amenity_us_rank"] = df_amenity["Natural Amenity"].rank(ascending=False, method="min").astype(int)

In [119]:
df_amenity["Natural Amenity_us_rank"]

0        922
1        517
2       1317
3       1562
4       1276
        ... 
3106     351
3107      87
3108     271
3109     777
3110    2075
Name: Natural Amenity_us_rank, Length: 3111, dtype: int64

In [120]:
df_amenity.to_csv("../data/df_amenity.csv")